In [ ]:
import numpy as np
import cv2

In [ ]:
dirname = "chessboards/"
img_names = [dirname + str(i) + ".jpg" for i in range(1,22)]

In [ ]:
square_size = 20.1 #mm
pattern_size = (8,5) # number of inner corner, (columns, rows) for OpenCV
indices = np.indices(pattern_size, dtype=np.float32)
indices *= square_size
coords_3D = (np.transpose(indices, [2, 1, 0])).reshape(-1, 2)
pattern_points = np.concatenate([coords_3D, np.zeros([coords_3D.shape[0], 1], dtype=np.float32)], axis=-1)

In [ ]:
def processImage(fn):
    img = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
    # Check image loaded correctly
    if img is None:
        print("Failed to load", fn)
        return None
    # Finding corners
    found, corners = cv2.findChessboardCorners(img, pattern_size)
    if found:
        # Refining corner position
        term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 5, 1)
        cv2.cornerSubPix(img, corners, (5, 5), (-1, -1), term)
    else:
        print('chessboard not found')
        return None
    return (corners.reshape(-1, 2), pattern_points)

In [ ]:
chessboards = [processImage(fn) for fn in img_names]

In [ ]:
chessboards = [x for x in chessboards if x is not None]

In [ ]:
# Creating the lists of 2D and 3D points 
obj_points = [] #3D points
img_points = [] #2D points

for (corners, pattern_points) in chessboards:
        img_points.append(corners)
        obj_points.append(pattern_points)

In [ ]:
# Getting the width and height of the images
h, w = cv2.imread(img_names[0], cv2.IMREAD_GRAYSCALE).shape[:2]

# Calibrating Camera
rms, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, (w, h), None, None)

rotation_matrix = cv2.Rodrigues(rvecs[0])[0]
translation_matrix = tvecs[0]
extrinsics_matrix = np.concatenate([rotation_matrix,translation_matrix], axis=1)

print("camera matrix:\n", camera_matrix)
print("distortion coefficients: ", dist_coefs.ravel())
print("Extrinsic matrix: ", extrinsics_matrix)